<a href="https://colab.research.google.com/github/sametgirgin/ColabNotebooks/blob/main/MergeDataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here’s the structure reflected in it:

- user_profiles

  - user_id (PK)
  - email
  - age

- user_health_data

  - user_id (FK → user_profiles)
  - date
  - average_heart_rate
  - average_glucose
  - sleep_hours
  - activity_level

- supplement_usage
  - user_id (FK → user_profiles)
  - date
  - supplement_name
  - dosage
  - dosage_unit
  - is_placebo
  - experiment_id (FK → experiments)

- experiments
  - experiment_id (PK)
  - name
  - description

**Relationships:**

user_profiles → user_health_data: one-to-many

user_profiles → supplement_usage: one-to-many

experiments → supplement_usage: one-to-many

In [1]:
import pandas as pd
import numpy as np

In [2]:
path ="https://raw.githubusercontent.com/sametgirgin/Data-Science-Notes/refs/heads/main/Data%20Engineering/Datasets/"

In [11]:
experiments_path = path + "experiments.csv"
user_health_path = path +"user_health_data.csv"
supplement_usage_path = path + "supplement_usage.csv"
user_profiles_path = path + "user_profiles.csv"

In [14]:
df_experiment = pd.read_csv(experiments_path)
df_user_health_data = pd.read_csv(user_health_path)
df_supplement_usage = pd.read_csv(supplement_usage_path)
df_user_profiles = pd.read_csv(user_profiles_path)

In [15]:
print(df_user_health_data.head())
print(df_supplement_usage.head())
print(df_user_profiles.head())
print(df_experiment.head())

   index                               user_id        date  \
0      0  c6ae338a-9f95-481c-a88d-24a58bc8fc71  2018-01-31   
1      1  c6ae338a-9f95-481c-a88d-24a58bc8fc71  2018-02-28   
2      2  c6ae338a-9f95-481c-a88d-24a58bc8fc71  2018-03-31   
3      3  c6ae338a-9f95-481c-a88d-24a58bc8fc71  2018-04-30   
4      4  5346f1dc-30f7-4e3a-9d35-eec6cb8835fa  2018-01-31   

   average_heart_rate  average_glucose sleep_hours  activity_level  
0           93.055612        70.089910        8.8h               1  
1           88.059964        78.411148        8.0H               3  
2           78.373746       107.418818       11.9h               1  
3           62.204061       117.259092        5.1h               1  
4           77.075789        99.221380        9.6h               2  
   index                               user_id        date supplement_name  \
0      0  38bbb850-2b41-47a2-b287-833317055c1a  2018-04-30       Vitamin C   
1      1  73420928-0991-41c2-9e31-930b18b61f95  2018-03-3

In [19]:
import pandas as pd
import numpy as np

def merge_all_data(user_health_path, supplement_usage_path, experiments_path, user_profiles_path):
    # Load datasets
    health = pd.read_csv(user_health_path, parse_dates=["date"])
    supplements = pd.read_csv(supplement_usage_path, parse_dates=["date"])
    experiments = pd.read_csv(experiments_path)
    profiles = pd.read_csv(user_profiles_path)

    # Drop the 'index' column from each dataframe
    health = health.drop('index', axis=1)
    supplements = supplements.drop('index', axis=1)
    experiments = experiments.drop('index', axis=1)
    profiles = profiles.drop('index', axis=1)

    # --- Step 1: Merge supplement_usage with experiments ---
    supplements = supplements.merge(experiments, on="experiment_id", how="left")

    # --- Step 2: Convert dosage to grams ---
    def convert_to_grams(row):
        if pd.isna(row["dosage"]):
            return np.nan
        if row["dosage_unit"].lower() == "mg":
            return row["dosage"] / 1000
        elif row["dosage_unit"].lower() == "g":
            return row["dosage"]
        else:
            return np.nan
    supplements["dosage_grams"] = supplements.apply(convert_to_grams, axis=1)

    # --- Step 3: Merge supplements + health ---
    merged = pd.merge(health, supplements, on=["user_id", "date"], how="outer")

    # --- Step 4: Add user profiles ---
    merged = merged.merge(profiles, on="user_id", how="left")

    # --- Step 5: Create user_age_group ---
    def age_group(age):
        if pd.isna(age): return "Unknown"
        elif age < 18: return "Under 18"
        elif 18 <= age <= 25: return "18-25"
        elif 26 <= age <= 35: return "26-35"
        elif 36 <= age <= 45: return "36-45"
        elif 46 <= age <= 55: return "46-55"
        elif 56 <= age <= 65: return "56-65"
        else: return "Over 65"
    merged["user_age_group"] = merged["age"].apply(age_group)

    # --- Step 6: Fill supplement_name ---
    merged["supplement_name"] = merged["supplement_name"].fillna("No intake")

    # --- Step 7: Select final columns ---
    final = merged[[
        "user_id", "date", "email", "user_age_group",
        "name", "supplement_name", "dosage_grams", "is_placebo",
        "average_heart_rate", "average_glucose", "sleep_hours", "activity_level"
    ]]

    # --- Step 8: Rename experiment_name ---
    final = final.rename(columns={"name": "experiment_name"})

    # --- Step 9: Ensure date is only YYYY-MM-DD ---
    final["date"] = pd.to_datetime(final["date"]).dt.strftime("%Y-%m-%d")

    # --- Step 10: Clean categorical/text data ---
    final["email"] = final["email"].str.strip().str.lower()
    final["supplement_name"] = final["supplement_name"].str.strip()
    final["experiment_name"] = final["experiment_name"].str.strip()

    # --- Step 11: Clean sleep_hours (strip h/H and convert to float) ---
    final["sleep_hours"] = (
        final["sleep_hours"]
        .astype(str)
        .str.replace("h", "", case=False, regex=True)
        .replace("nan", np.nan)
        .astype(float)
    )

    # --- Step 12: Convert numeric fields ---
    numeric_fields = ["dosage_grams", "average_heart_rate", "average_glucose", "activity_level"]
    for col in numeric_fields:
        final[col] = pd.to_numeric(final[col], errors="coerce")

    # --- Step 13: Convert is_placebo to boolean ---
    if final["is_placebo"].dtype != bool:
        final["is_placebo"] = final["is_placebo"].map({True: True, False: False, "true": True, "false": False, 1: True, 0: False})

    return final

In [21]:
merged_df = merge_all_data(user_health_path, supplement_usage_path, experiments_path, user_profiles_path)
display(merged_df.head())

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-01-31,user_65@myemail.com,26-35,Memory,Placebo,0.170989,False,84.172177,96.814856,11.4,2
1,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-02-28,user_65@myemail.com,26-35,Sleep Quality,Magnesium,0.494938,False,85.200747,130.836935,5.8,2
2,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-03-31,user_65@myemail.com,26-35,Sleep Quality,Placebo,0.184596,False,76.695318,109.782341,7.0,1
3,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-04-30,user_65@myemail.com,26-35,Endurance,Omega-3,0.313256,True,65.772482,100.765545,7.5,1
4,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-01-31,contact_390@email.com,36-45,NaN,No intake,NaN,NaN,86.762141,103.689177,8.2,2


In [22]:
print(merged_df.info())
print(merged_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2721 entries, 0 to 2720
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             2721 non-null   object 
 1   date                2721 non-null   object 
 2   email               2721 non-null   object 
 3   user_age_group      2721 non-null   object 
 4   experiment_name     2000 non-null   object 
 5   supplement_name     2721 non-null   object 
 6   dosage_grams        2000 non-null   float64
 7   is_placebo          2000 non-null   object 
 8   average_heart_rate  2721 non-null   float64
 9   average_glucose     2721 non-null   float64
 10  sleep_hours         2721 non-null   float64
 11  activity_level      2721 non-null   int64  
dtypes: float64(4), int64(1), object(7)
memory usage: 255.2+ KB
None
user_id                 0
date                    0
email                   0
user_age_group          0
experiment_name       721
suppleme